In [41]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

In [42]:
def load_data(path):
  return pd.read_csv(path)

def filter_columns(data, year):
  if year <= 2022:
    data.rename(columns={'Sample Size': 'Population',
                         'Question Number': 'Question',
                         'Provider': 'Provider name',
                         'Subject Code': 'Subject code',
                         'Response': 'Responses',
                         'Answered 1': 'Option 5',
                         'Answered 2': 'Option 4',
                         'Answered 3': 'Option 3',
                         'Answered 4': 'Option 2',
                         'Answered 5': 'Option 1',
                         'Actual value': 'Positivity'}, inplace=True)
  else:
    data.rename(columns={'Positivity measure (%)': 'Positivity', 'Publication response rate (%)': 'Response Rate'}, inplace=True)
  data = data[['UKPRN', 'Provider name', 'Subject code', 'Subject', 'Question', 'Responses', 'Population',
  'Option 1', 'Option 2', 'Option 3', 'Option 4', 'Option 5', 'Positivity']]
  return data

def drop_theme_rows(data, year):
  if year <= 2022: term = 'Scale'
  else: term = 'Theme'
  theme_df = data[data['Question'].str.contains(term)]
  theme_indices = list(theme_df.index.values) 
  data = data[~data.index.isin(theme_indices)]
  hc_df = data[data['Question'].str.contains('HC')]
  hc_indices = list(hc_df.index.values) 
  data = data[~data.index.isin(hc_indices)]
  return data

# get first 3 characters of the string - Q01, Q27
def get_qn_num(data, year):
  if year > 2022:
    data[['Question Num', 'Question']] = data['Question'].str.split(':', expand=True)
  return data

def drop_null_surveys(data):
  null_prn = list(data[(data['Option 1'].isna())]['UKPRN'].unique())
  null_course = list(data[(data['Option 1'].isna())]['Subject code'].unique())
  if len(null_prn) == len(null_course):
    for i in range(len(null_prn)):
      data = data[~((data['UKPRN'] == null_prn[i]) & (data['Subject code'] == null_course[i]))]
  return data

In [43]:
path = 'data/2023/NSS3_2023.csv'
# path = 'data/2022/level_3.csv'
year = int(path.split('/')[1])

# df = load_data(path)
# df = df.pipe(filter_columns, df, year).pipe(drop_theme_rows, df, year).pipe(get_qn_num, df, year).pipe(drop_null_surveys, df)

def pipeline(*funcs):
    def inner(data):
        result = data
        for func in funcs:
            result = func(result)
        return result
    return inner

data_pipeline = pipeline(
    lambda x: load_data(x),
    lambda x: filter_columns(x, year),
    lambda x: drop_theme_rows(x, year),
    lambda x: get_qn_num(x, year),
    lambda x: drop_null_surveys(x)
)

df = data_pipeline(path)

df.head()

,UKPRN,Provider name,Subject code,Subject,Question,Responses,Population,Option 1,Option 2,Option 3,Option 4,Option 5,Positivity
0,10007783,University of Aberdeen,CAH01-01-02,Medicine (non-specific),Q01,118,162,44%,49%,3%,4%,0%,93
1,10007783,University of Aberdeen,CAH01-01-02,Medicine (non-specific),Q02,118,162,39%,53%,6%,3%,0%,92
2,10007783,University of Aberdeen,CAH01-01-02,Medicine (non-specific),Q03,118,162,69%,25%,4%,0%,2%,94
3,10007783,University of Aberdeen,CAH01-01-02,Medicine (non-specific),Q04,118,162,55%,37%,4%,3%,1%,92
4,10007783,University of Aberdeen,CAH01-01-02,Medicine (non-specific),Q05,117,162,41%,49%,7%,2%,2%,90


In [44]:
import dill

data_pipeline = pipeline(
    lambda x: load_data(x),
    lambda x: filter_columns(x, year),
    lambda x: drop_theme_rows(x, year),
    lambda x: get_qn_num(x, year),
    lambda x: drop_null_surveys(x)
)

with open('../models/data_pipeline.pkl', 'wb') as f:
    dill.dump(data_pipeline, f)

In [45]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 165024 entries, 0 to 165023
Data columns (total 13 columns):
 #   Column         Non-Null Count   Dtype 
---  ------         --------------   ----- 
 0   UKPRN          165024 non-null  int64 
 1   Provider name  165024 non-null  object
 2   Subject code   165024 non-null  object
 3   Subject        165024 non-null  object
 4   Question       165024 non-null  object
 5   Responses      165024 non-null  int64 
 6   Population     165024 non-null  int64 
 7   Option 1       165024 non-null  object
 8   Option 2       165024 non-null  object
 9   Option 3       165024 non-null  object
 10  Option 4       165024 non-null  object
 11  Option 5       165024 non-null  object
 12  Positivity     165024 non-null  int64 
dtypes: int64(4), object(9)
memory usage: 16.4+ MB


In [46]:
df.head()

,UKPRN,Provider name,Subject code,Subject,Question,Responses,Population,Option 1,Option 2,Option 3,Option 4,Option 5,Positivity
0,10007783,University of Aberdeen,CAH01-01-02,Medicine (non-specific),Q01,118,162,44%,49%,3%,4%,0%,93
1,10007783,University of Aberdeen,CAH01-01-02,Medicine (non-specific),Q02,118,162,39%,53%,6%,3%,0%,92
2,10007783,University of Aberdeen,CAH01-01-02,Medicine (non-specific),Q03,118,162,69%,25%,4%,0%,2%,94
3,10007783,University of Aberdeen,CAH01-01-02,Medicine (non-specific),Q04,118,162,55%,37%,4%,3%,1%,92
4,10007783,University of Aberdeen,CAH01-01-02,Medicine (non-specific),Q05,117,162,41%,49%,7%,2%,2%,90


In [47]:
# df['index_rn'] = df.index
# data_28 = df[df['Question Num']=='Q28']
# for i in range(len(df['Option 1'])):
#   if df['index_rn'].iloc[i] in list(data_28['index_rn'].values):
#     df.iloc[i, df.columns.get_loc('Option 3')] = df['Option 4'].iloc[i]
#     df.iloc[i, df.columns.get_loc('Option 4')] = df['Option 5'].iloc[i]
# # df[df['Question Num'] == 'Q28'].head()

KeyError: 'Question Num'

In [ ]:
# df['Diff'] = df['Option 4'] - df['Option 5']
# df['Diff'].describe()
# df.drop(columns=['Option 5', 'Diff', 'index_rn'], inplace=True)

In [ ]:
def get_index_col(data):
  data.index = np.arange(1, len(data) + 1)
  data['index1'] = data.index
  return data  

def get_qn27_data(data):
  q27_rows = data[data['Question Num'] == 'Q27']['index1'].values.tolist()
  with_27qn = pd.DataFrame(columns=list(data.columns))
  for i in range(len(q27_rows)):
    new_data_27 = data.iloc[q27_rows[i]-27:q27_rows[i]]
    with_27qn = pd.concat([with_27qn, new_data_27], ignore_index=True)
  return with_27qn

def get_qn28_data(data):
  q28_rows = data[data['Question Num'] == 'Q28']['index1'].values.tolist()
  with_28qn = pd.DataFrame(columns=list(data.columns))
  for i in range(len(q28_rows)):
    new_data_28 = data.iloc[q28_rows[i]-27:q28_rows[i]]
    with_28qn = pd.concat([with_28qn, new_data_28], ignore_index=True)
  return with_28qn

def generate_subsets(data):
  data = get_index_col(data)
  english_final = get_qn27_data(data)
  not_english_final = get_qn28_data(data)
  drop_rows_1 = not_english_final['index1'].values.tolist()
  drop_rows_2 = english_final['index1'].values.tolist()
  uk_final = data.copy()
  uk_final = data[~(data['index1'].isin(drop_rows_1))]
  uk_final = uk_final[~(uk_final['index1'].isin(drop_rows_2))]
  return english_final, not_english_final, uk_final

# Concat to empty df deprecated
# Add rows first time, check if rows 0, if not, concat other sets

In [ ]:
english_final, not_english_final, uk_final = generate_subsets(df)

C:\Users\pm_\AppData\Local\Temp\ipykernel_17488\2712263847.py:11: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  with_27qn = pd.concat([with_27qn, new_data_27], ignore_index=True)
C:\Users\pm_\AppData\Local\Temp\ipykernel_17488\2712263847.py:19: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  with_28qn = pd.concat([with_28qn, new_data_28], ignore_index=True)


In [ ]:
print(english_final.shape)
print(not_english_final.shape)
print(uk_final.shape)

(294732, 15)
(78111, 15)
(15938, 15)


In [ ]:
english_final.loc[:, 'Region'] = ['English']*len(english_final['Option 1'].values)
not_english_final.loc[:, 'Region'] = ['Non-English']*len(not_english_final['Option 1'].values)
uk_final.loc[:, 'Region'] = ['UK']*len(uk_final['Option 1'].values)

In [ ]:
not_english_final.to_csv('data/2023/generated/pre-train/not_english_final.csv', index=False)
english_final.to_csv('data/2023/generated/pre-train/english_final.csv', index=False)
uk_final.to_csv('data/2023/generated/pre-train/uk_final.csv', index=False)

In [ ]:
pdList = [not_english_final, uk_final, english_final]
combined_df = pd.concat(pdList)

In [ ]:
combined_df.to_csv('data/2023/generated/pre-train/combined_df.csv', index=False)

In [ ]:
df = combined_df.copy()

In [ ]:
def get_processed_data(data):
  sample = data.copy()
  # sample = data[['Responses', 'Option 1', 'Option 2', 'Option 3', 'Option 4', 'Option 5', 'Positivity', 'Question Num']]
  sample = sample[['Responses', 'Option 1', 'Option 2', 'Option 3', 'Option 4', 'Positivity', 'Question Num', 'Region']]
  sample.loc[:, 'Opt-1'] = round((sample['Option 1'] / sample['Responses']) * 100, 1)
  sample.loc[:, 'Opt-2'] = round((sample['Option 2'] / sample['Responses']) * 100, 1)
  sample.loc[:, 'Opt-3'] = round((sample['Option 3'] / sample['Responses']) * 100, 1)
  sample.loc[:, 'Opt-4'] = round((sample['Option 4'] / sample['Responses']) * 100, 1)
  only_options = sample[['Question Num', 'Opt-1', 'Opt-2', 'Opt-3', 'Opt-4', 'Region']]
  # region_false = ''
  # if region == 'English': region_false = 'R_Non-English'
  # else: region_false = 'R_English'
  # only_options[region] = [region]*len(only_options['Opt-1'])
  only_options = only_options.copy()
  only_options.loc[:, 'Positivity'] = only_options['Opt-1'] + only_options['Opt-2']
  # data_2 = pd.get_dummies(only_options, prefix=['N'], dtype=float)
  # data_2[region_false] = [0.0]*len(data_2)
  return only_options

In [ ]:
only_options = get_processed_data(df)
only_options.head()

,Question Num,Opt-1,Opt-2,Opt-3,Opt-4,Region,Positivity
0,Q01,25.0,64.8,7.6,2.5,Non-English,89.8
1,Q02,20.8,62.7,14.4,2.1,Non-English,83.5
2,Q03,64.4,32.6,2.5,0.4,Non-English,97.0
3,Q04,46.2,42.4,10.6,0.8,Non-English,88.6
4,Q05,39.3,46.2,12.4,2.1,Non-English,85.5
